<html><head><meta content="text/html; charset=UTF-8" http-equiv="content-type"><style type="text/css">ol</style></head><body class="c5"><p class="c0 c4"><span class="c3"></span></p><p class="c2 title" id="h.rrbabt268i6e"><h1>CaImAn&rsquo;s Demo pipeline</h1></p><p class="c0"><span class="c3">This notebook will help to demonstrate the process of CaImAn and how it uses different functions to denoise, deconvolve and demix neurons from a Calcium Imaging Video. </span></p>
<p><img src="docs/img/quickintro.png" /></p>
<p class="c0"><span class="c3">More information can be found in CaImAn&rsquo;s documentation. </span></p>
</html>



In [6]:
#!/usr/bin/env python
from __future__ import division
from __future__ import print_function
from builtins import zip
from builtins import str
from builtins import map
from builtins import range
from past.utils import old_div
try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')    
except:
    print('Not IPYTHON')
import numpy as np
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib qt')   
import caiman as cm
from caiman.source_extraction import cnmf
from caiman.utils.utils import download_demo
from caiman.utils.visualization import inspect_correlation_pnr
from caiman.components_evaluation import estimate_components_quality_auto
from caiman.motion_correction import motion_correct_oneP_rigid
import os
import cv2
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour

try:
    cv2.setNumThreads(1)
except:
    print('Open CV is naturally single threaded')
import bokeh.plotting as bpl
bpl.output_notebook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

### Setup some parameters
many of them will be set directly calling the CNMF object

In [7]:
fnames = ['data_endoscope.tif']
frate = 10 # movie frame rate
gSig = 3   # gaussian width of a 2D gaussian kernel, which approximates a neuron
gSiz = 10  # average diameter of a neuron
do_motion_correction = True

Download the dataset if not already present

In [8]:
base_folder = './example_movies/'  
download_demo(fnames[0])
fnames = [os.path.abspath(os.path.join(base_folder,fnames[0]))]

File already downloaded


### (Re)start cluster.  

In [9]:
try:
    dview.terminate() # stop it if it was running
except:
    pass

c, dview, n_processes = cm.cluster.setup_cluster(backend='local', # use this one
                                                 n_processes=16,  # number of process to use, if you go out of memory try to reduce this one
                                                 )

0
0
0
0
0
0
0
0
10
10
0
0
10
10
10
10
20
20
10
20
10
20
30
20
20
20
10
10
30
30
40
30
20
30
30
20
20
30
40
40
40
50
40
30
40
30
50
30
40
50
50
50
60
60
40
40
50
40
50
70
60
60
60
50
70
50
50
80
60
60
60
90
70
70
80
60
70
60
70
70
70
80
90
80
70
80
70
80
80
80
90
90
90
80
90
80
90
90
90
90
0
0
0
10
0
0
0
10
20
10
0
10
10
20
30
10
20
20
40
0
0
20
0
10
20
30
30
30
50
10
10
40
30
30
10
20
40
40
60
20
20
50
50
30
40
20
40
70
50
30
60
30
60
40
80
30
50
50
70
60
40
40
70
90
60
50
50
40
80
60
70
50
70
80
60
60
90
50
70
80
60
80
90
70
60
70
80
90
70
90
70
80
80
80
90
80
90
90
90
90
0
0
10
0
0
20
10
10
10
0
30
0
20
0
40
20
10
20
10
0
30
50
10
20
30
20
30
0
0
10
60
40
20
30
30
40
40
10
10
30
40
40
70
20
50
20
50
50
50
20
50
40
80
30
60
60
60
60
30
90
50
30
60
40
70
70
70
60
40
70
40
80
70
50
80
70
80
90
50
50
90
80
80
80
60
60
90
60
90
90
70
90
70
70
80
90
80
80
90
90
0
0
10
0
10
0
10
0
0
20
0
20
0
10
10
20
10
30
10
30
10
20
20
20
40
30
20
30
40
20
30
30
50
40
0
40
30
50
0
30
40
40
10
60
50
40
10

/home/fpbatta/src/extern/CaImAn_stable/caiman/source_extraction/cnmf/deconvolution.py:1019: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  g = np.linalg.lstsq(A, xc[lags + 1:])[0]


0 neurons have been initialized


/home/fpbatta/src/extern/CaImAn_stable/caiman/source_extraction/cnmf/deconvolution.py:1019: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  g = np.linalg.lstsq(A, xc[lags + 1:])[0]
/home/fpbatta/src/extern/CaImAn_stable/caiman/source_extraction/cnmf/deconvolution.py:1019: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  g = np.linalg.lstsq(A, xc[lags + 1:])[0]


0 neurons have been initialized
0 neurons have been initialized


...

parallel dot product block size: 1000
Start product
Transposing
999
1999
2999
3999
4999
5999
6479


/home/fpbatta/src/extern/CaImAn_stable/caiman/source_extraction/cnmf/initialization.py:1100: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  f_in = np.linalg.lstsq(b_in, B)[0]


parallel dot product block size: 1000
Start product
Transposing
999
1999
2999
3999
4999
5999
6399
4999
9999
14999
16383


/home/fpbatta/src/extern/CaImAn_stable/caiman/source_extraction/cnmf/deconvolution.py:1019: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  g = np.linalg.lstsq(A, xc[lags + 1:])[0]
/home/fpbatta/src/extern/CaImAn_stable/caiman/source_extraction/cnmf/deconvolution.py:1019: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  g = np.linalg.lstsq(A, xc[lags + 1:])[0]
/home/fpbatta/src/extern/CaImAn_stable/caiman/source_extraction/cnmf/deconvolution.py:1019: FutureWarning: `rcond` parameter will change to the default of machine pre

4999
9999
16383
14999
4999
9999
14999
16383
tB:-2.0,tA:9.0
Computing event exceptionality delta
tB:-2.0,tA:9.0
Computing event exceptionality delta
tB:-2.0,tA:9.0
Computing event exceptionality delta
Removing Baseline
binning data ...
interpolating data ...
(5, 5)
Computing event exceptionality
Evaluating spatial footprint
Removing Baseline
binning data ...
Removing Baseline
interpolating data ...
binning data ...
(5, 50)
interpolating data ...
(5, 50)
Computing event exceptionality
components evaluated:0
Computing event exceptionality
Evaluating spatial footprint
Evaluating spatial footprint
components evaluated:0
Neuron:0 includes overlaping spiking neurons
Neuron:3 includes overlaping spiking neurons
components evaluated:0
Neuron:3 includes overlaping spiking neurons
Neuron:5 includes overlaping spiking neurons
Neuron:26 includes overlaping spiking neurons


### Creation of a memory mappable file. 
    - Performs motion correction and simultaneously creates a memory mappable file in F order
    - Transforms into C order (much more efficient for parallel processing
    - If you have multiple files there are ways to process many at the same time (not shown)
    

In [15]:
if do_motion_correction:
    mc = motion_correct_oneP_rigid(fnames[0],                        # name of file to motion correct
                               gSig_filt = [gSig]*2,                 # size of filter, xhange this one if algorithm does not work 
                               max_shifts = [5,5],                   # maximum shifts allowed in each direction 
                               dview=dview, 
                               splits_rig = 10,                      # number of chunks for parallelizing motion correction (remember that it should hold that length_movie/num_splits_to_process_rig>100) 
                               save_movie = True)                    # whether to save movie in memory mapped format
    
    new_templ = mc.total_template_rig
    plt.figure(figsize=(15,8))
    plt.subplot(1,2,1)    
    plt.title('Filtered template')
    plt.imshow(new_templ)       #% plot template
    plt.subplot(1,2,2)
    plt.title('Estimated shifts')
    plt.plot(mc.shifts_rig)     #% plot rigid shifts
    plt.legend(['x shifts', 'y shifts'])
    plt.xlabel('frames')
    plt.ylabel('pixels')
    
    bord_px_rig = np.ceil(np.max(mc.shifts_rig)).astype(np.int)     #borders to eliminate from movie because of motion correction        
    fname_new = cm.save_memmap(mc.fname_tot_rig, base_name='memmap_', order = 'C') # transforming memoruy mapped file in C order (efficient to perform computing)
else:
    #% create memory mappable file
    fname_new = cm.save_memmap(fnames, base_name='memmap_', order = 'C')

# load memory mappable file
Yr, dims, T = cm.load_memmap(fname_new)
Y = Yr.T.reshape((T,) + dims, order='F')

Rigid Motion Correction
5.3534045
-1.699735164642334
Frame 100
cubic interpolation
Frame 100
-1.699735164642334
Frame 100
cubic interpolation
Frame 100


/home/fpbatta/src/extern/CaImAn_stable/caiman/base/movies.py:260: UserWarning: ** Pixels averages are too negative. Removing 1 percentile. **
  '** Pixels averages are too negative. Removing 1 percentile. **')
/home/fpbatta/src/extern/CaImAn_stable/caiman/base/movies.py:279: UserWarning: Pixels averages are too negative for template. Removing 1 percentile.
  'Pixels averages are too negative for template. Removing 1 percentile.')


-1.699735164642334
Frame 100
cubic interpolation
Frame 100
Adding to movie 5.3534045
0
saving!
** Startting parallel motion correction **
** Finished parallel motion correction **
0.045475096
/home/fpbatta/src/extern/CaImAn_stable/example_movies/data_endoscope_rig__d1_128_d2_128_d3_1_order_F_frames_1000_.mmap
<class 'str'>
/home/fpbatta/src/extern/CaImAn_stable/example_movies/data_endoscope_rig__d1_128_d2_128_d3_1_order_F_frames_1000_.mmap
loading in memory
mmap


Play the movie (optional). This will require loading the movie in memory which in general is not needed by the pipeline. Displaying the movie uses the OpenCV library. Press `q` to close the video panel. **BEWARE** the movie may appear in the background!

In [16]:
m_orig = cm.movie(Y)
downsample_ratio = 1.
offset_mov = -np.min(m_orig[:100])  # make the dataset mostly non-negative
m_orig.resize(1, 1, downsample_ratio).play(
gain=2, offset=offset_mov, fr=30, magnification=2)

### Inspect summary images and set parameters
Check the optimal values of min_corr and min_pnr by moving slider in the figure that pops up

In [17]:
# compute some summary images (correlation and peak to noise)
cn_filter, pnr = cm.summary_images.correlation_pnr(Y, gSig=gSig, swap_dim=False) # change swap dim if output looks weird, it is a problem with tiffile
# inspect the summary images and set the parameters
inspect_correlation_pnr(cn_filter,pnr)


In [18]:

min_corr = .8 # min correlation of peak (from correlation image)
min_pnr = 10 # min peak to noise ratio
min_SNR = 3 # adaptive way to set threshold on the transient size
r_values_min = 0.85  # threshold on space consistency (if you lower more components will be accepted, potentially with worst quality)
decay_time = 0.4  #decay time of transients/indocator

In [19]:
### Set CNMF parameters and run it

In [20]:
cnm = cnmf.CNMF(n_processes=n_processes, 
                method_init='corr_pnr',                 # use this for 1 photon
                k=70,                                   # neurons per patch
                gSig=(3, 3),                            # half size of neuron
                gSiz=(10, 10),                          # in general 3*gSig+1
                merge_thresh=.8,                        # threshold for merging
                p=1,                                    # order of autoregressive process to fit
                dview=dview,                            # if None it will run on a single thread
                tsub=2,                                 # downsampling factor in time for initialization, increase if you have memory problems             
                ssub=2,                                 # downsampling factor in space for initialization, increase if you have memory problems
                Ain=None,                               # if you want to initialize with some preselcted components you can pass them here as boolean vectors
                rf=(40, 40),                            # half size of the patch (final patch will be 100x100)
                stride=(20, 20),                        # overlap among patches (keep it at least large as 4 times the neuron size)
                only_init_patch=True,                   # just leave it as is
                gnb=16,                                 # number of background components
                nb_patch=16,                            # number of background components per patch
                method_deconvolution='oasis',           #could use 'cvxpy' alternatively
                low_rank_background=True,               #leave as is
                update_background_components=True,      # sometimes setting to False improve the results
                min_corr=min_corr,                      # min peak value from correlation image 
                min_pnr=min_pnr,                        # min peak to noise ration from PNR image
                normalize_init=False,                   # just leave as is
                center_psf=True,                        # leave as is for 1 photon
                del_duplicates=True)                    # whether to remove duplicates from initialization

cnm.fit(Y)

(1000, 128, 128)
using 16 processes
using 4000 pixels per process
using 5000 block_size
(80, 80)
17.184683799743652
Transforming patches into full matrix
Skipped %d Empty Patch 0
Generating background
Compressing background components with a low rank NMF
Generating background DONE
merging
[30 31]
[81 98]


/home/fpbatta/anaconda3/envs/caiman/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)
/home/fpbatta/src/extern/CaImAn_stable/caiman/source_extraction/cnmf/deconvolution.py:1019: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  g = np.linalg.lstsq(A, xc[lags + 1:])[0]


No neurons merged!
update temporal
Generating residuals
parallel dot product block size: 5000
Start product
Processed:[0, 4]
Transposing
entering the deconvolution 
28 out of total 105 temporal components updated
51 out of total 105 temporal components updated
67 out of total 105 temporal components updated
82 out of total 105 temporal components updated
95 out of total 105 temporal components updated
103 out of total 105 temporal components updated
105 out of total 105 temporal components updated
28 out of total 105 temporal components updated
51 out of total 105 temporal components updated
67 out of total 105 temporal components updated
82 out of total 105 temporal components updated
95 out of total 105 temporal components updated
103 out of total 105 temporal components updated
105 out of total 105 temporal components updated
stopping: overall temporal component not changing significantly
update spatial ...
Initializing update of Spatial Components
computing the distance indicators


plot contours of identified components against correlation image

In [21]:
crd = cm.utils.visualization.plot_contours(cnm.A, cn_filter, thr=.8, vmax=0.95)

/home/fpbatta/src/extern/CaImAn_stable/caiman/utils/visualization.py:941: UserWarning: The way to call utilities.plot_contours has changed. Look at the definition for more details.
  warn("The way to call utilities.plot_contours has changed. Look at the definition for more details.")


## Component Evaluation

The processing in patches creates several spurious components. These are filtered out by evaluating each component using three different criteria:

- the shape of each component must be correlated with the data at the corresponding location within the FOV
- a minimum peak SNR is required over the length of a transient
- each shape passes a CNN based classifier

<img src="docs/img/evaluationcomponent.png"/>

In [22]:
#%% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier

idx_components, idx_components_bad, comp_SNR, r_values, pred_CNN = estimate_components_quality_auto(
                            Y, cnm.A, cnm.C, cnm.b, cnm.f, cnm.YrA, frate, 
                            decay_time, gSig, dims, dview = dview, 
                            min_SNR=min_SNR, r_values_min = r_values_min, min_std_reject = 0.5, use_cnn = False)

print(' ***** ')
print((len(cnm.C)))
print((len(idx_components)))
print(r_values[idx_components_bad])

EVALUATING IN PARALLEL... NOT RETURNING ERFCs
 ***** 
105
91
[ 0.44978039 -0.22632325  0.03454946  0.36198095  0.06155593 -0.01481715
  0.39492054 -0.05129989  0.11592654  0.40387176  0.39647351 -0.27337794
  0.20040096 -0.49295236]


Plot contours of selected and rejected components

In [23]:
#%% PLOT COMPONENTS

plt.figure(figsize=(15,8));
plt.subplot(121);
crd = cm.utils.visualization.plot_contours(cnm.A.tocsc()[:,idx_components], cn_filter, thr=.8, vmax=0.95)
plt.title('Contour plots of accepted components')
plt.subplot(122); 
crd = cm.utils.visualization.plot_contours(cnm.A.tocsc()[:,idx_components_bad], cn_filter, thr=.8, vmax=0.95)
plt.title('Contour plots of rejected components')

/home/fpbatta/src/extern/CaImAn_stable/caiman/utils/visualization.py:941: UserWarning: The way to call utilities.plot_contours has changed. Look at the definition for more details.
  warn("The way to call utilities.plot_contours has changed. Look at the definition for more details.")


Text(0.5,1,'Contour plots of rejected components')

View traces of accepted and rejected components. Note that if you get data rate error you can start Jupyter notebooks using:
'jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10'

In [24]:
# accepted components
nb_view_patches(Yr, cnm.A.tocsc()[:, idx_components], cnm.C[idx_components], 
                cnm.b, cnm.f, dims[0], dims[1], YrA=cnm.YrA[idx_components], image_neurons = cn_filter,
                denoised_color = 'red', thr=0.8, cmap='gray');

In [25]:
# rejected components
nb_view_patches(Yr, cnm.A.tocsc()[:, idx_components_bad], cnm.C[idx_components_bad], 
                cnm.b, cnm.f, dims[0], dims[1], YrA=cnm.YrA[idx_components_bad], image_neurons = cn_filter,
                denoised_color = 'red', thr=0.8, cmap='gray');

### Stop cluster

In [26]:
#%%
cm.stop_server(dview=dview)

 done


### Some instructive movies

In [27]:
#%% denoised movie
cm.movie(np.reshape(cnm.A.tocsc()[:,idx_components].dot(cnm.C[idx_components])+cnm.b.dot(cnm.f),dims+(-1,), order = 'F').transpose(2,0,1)).play(magnification=3, gain = 1.)

In [28]:
#%% only neurons
cm.movie(np.reshape(cnm.A.tocsc()[:,idx_components].dot(cnm.C[idx_components]),dims+(-1,), order = 'F').transpose(2,0,1)).play(magnification=3, gain = 10.)

In [29]:
#%% only the background
cm.movie(np.reshape(cnm.b.dot(cnm.f),dims+(-1,), order = 'F').transpose(2,0,1)).play(magnification=3, gain = 1.)

In [30]:
#%% residuals
cm.movie(np.array(Y)-np.reshape(cnm.A.tocsc()[:,:].dot(cnm.C[:])+cnm.b.dot(cnm.f),dims+(-1,), order = 'F').transpose(2,0,1)).play(magnification=3, gain = 10., fr = 10)


In [31]:
#%% eventuall, you can rerun the algorithm on the residuals
plt.imshow(cm.movie(np.array(Y)-np.reshape(cnm.A.tocsc()[:,:].dot(cnm.C[:])+cnm.b.dot(cnm.f),dims+(-1,), order = 'F').transpose(2,0,1)).local_correlations(swap_dim=False))